In [1]:
pip install flaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.3/313.3 kB 6.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from flaml import autogen
from flaml import AutoML
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split


2024-10-20 00:40:39,207	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-10-20 00:40:40,976	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
import pandas as pd

In [4]:
df=pd.read_csv("/kaggle/input/deepflow-encryption-dataset/encryption_attack_dataset.csv")

In [5]:
df

,Algorithm,Key Size,Encryption Time,Decryption Time,Attack Type,Attack Success,Attack Time,Next Algo,Next KSize
0,AES,128,0.005115,0.000095,Classical Attack,False,10.000000,NaN,NaN
1,Kyber,2048,0.001461,0.002356,Quantum Attack,True,3.476478,McEliece,Higher
2,McEliece,192,0.000316,0.000113,Classical Attack,False,10.000000,NaN,NaN
3,RC4,128,0.000236,0.000018,Quantum Attack,True,1.737776,Kyber,Higher
4,AES,128,0.000048,0.000037,Classical Attack,False,10.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3995,RC4,128,0.000023,0.000014,Classical Attack,False,10.000000,NaN,NaN
3996,AES,128,0.000041,0.000029,Quantum Attack,True,3.800863,Kyber,Higher
3997,Kyber,2048,0.000886,0.002002,Classical Attack,True,1.821394,McEliece,Higher
3998,McEliece,192,0.000167,0.000104,Quantum Attack,True,2.869599,Kyber,Higher


In [6]:
df['Next Algo']= df['Next Algo'].fillna("None")


In [7]:
df['Next Algo'].value_counts()

Next Algo
None        2358
McEliece     877
Kyber        765
Name: count, dtype: int64

In [8]:
X=df[['Algorithm', "Key Size","Attack Type", "Attack Success", "Attack Time"]]
y=df['Next Algo']

In [9]:
y

0           None
1       McEliece
2           None
3          Kyber
4           None
          ...   
3995        None
3996       Kyber
3997    McEliece
3998       Kyber
3999        None
Name: Next Algo, Length: 4000, dtype: object

In [10]:
X_train, X_test, y_train, y_test= train_test_split(X, y,test_size=0.2, stratify=y, random_state=66 )

In [11]:
X_train

,Algorithm,Key Size,Attack Type,Attack Success,Attack Time
1840,AES,128,Classical Attack,False,10.000000
1611,RC4,128,Classical Attack,False,10.000000
1947,RC4,128,Quantum Attack,True,1.897266
3057,Kyber,2048,Classical Attack,False,10.000000
2600,AES,128,Classical Attack,False,10.000000
...,...,...,...,...,...
2788,AES,128,Classical Attack,True,2.041900
899,RC4,128,Quantum Attack,True,2.854953
1433,Kyber,2048,Quantum Attack,False,10.000000
1744,AES,128,Quantum Attack,True,4.308194


In [12]:
automl = AutoML()

automl.fit(X_train, y_train, time_budget=60)  




[flaml.automl.logger: 10-20 00:40:54] {1728} INFO - task = classification
[flaml.automl.logger: 10-20 00:40:54] {1739} INFO - Evaluation method: cv
[flaml.automl.logger: 10-20 00:40:54] {1838} INFO - Minimizing error metric: log_loss
[flaml.automl.logger: 10-20 00:40:54] {1955} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'sgd', 'catboost', 'lrl1']
[flaml.automl.logger: 10-20 00:40:54] {2258} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 10-20 00:40:55] {2393} INFO - Estimated sufficient time budget=1911s. Estimated necessary time budget=47s.
[flaml.automl.logger: 10-20 00:40:55] {2442} INFO -  at 0.5s,	estimator lgbm's best error=0.5827,	best estimator lgbm's best error=0.5827
[flaml.automl.logger: 10-20 00:40:55] {2258} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 10-20 00:40:55] {2442} INFO -  at 0.6s,	estimator lgbm's best error=0.5827,	best estimator lgbm's best error=0.5827
[flaml.automl.log

In [13]:
print("Best Model:", automl.best_estimator)
print("Best Loss on validation set:", automl.best_loss)

Best Model: rf
Best Loss on validation set: 0.19097311113873888


In [14]:
test_accuracy = automl.score(X_test, y_test)
print("Test accuracy:", test_accuracy)

Test accuracy: 0.85375


In [15]:
model=automl
filename = 'encryption_QPC_12.pkl'


In [16]:
with open(filename, 'wb') as file:
    pickle.dump(model, file)


In [17]:
with open(filename, 'rb') as file:
    loaded_model = pickle.load(file)

In [18]:

test_accuracy = loaded_model.score(X_test, y_test)


In [19]:
test_accuracy

0.85375